In [65]:
import json
import requests
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error

ImportError: Traceback (most recent call last):
  File "/home/nitin/.local/lib/python3.7/site-packages/tensorflow/python/__init__.py", line 61, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/home/nitin/.local/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/home/nitin/.local/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/usr/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: /home/nitin/.local/lib/python3.7/site-packages/tensorflow/python/_pywrap_tensorflow.so: invalid ELF header


Failed to load the native TensorFlow runtime.

See https://github.com/tensorflow/tensorflow/blob/master/tensorfl

In [56]:
sampleName = ["Bitstamp_BTCUSD_","_minute.csv"]

csvfiles =  []

for i in range(2015,2023):
    csvfiles.append(str(i).join(sampleName))

csvfiles

['Bitstamp_BTCUSD_2015_minute.csv',
 'Bitstamp_BTCUSD_2016_minute.csv',
 'Bitstamp_BTCUSD_2017_minute.csv',
 'Bitstamp_BTCUSD_2018_minute.csv',
 'Bitstamp_BTCUSD_2019_minute.csv',
 'Bitstamp_BTCUSD_2020_minute.csv',
 'Bitstamp_BTCUSD_2021_minute.csv',
 'Bitstamp_BTCUSD_2022_minute.csv']

In [57]:
df = pd.read_csv(csvfiles[0])
pd.DataFrame(df).shape
df

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1451606340,2015-12-31 23:59:00,BTC/USD,430.89,430.89,430.89,430.89,0.000000,0.000000
1,1451606280,2015-12-31 23:58:00,BTC/USD,429.77,430.89,429.77,430.89,0.219208,94.454380
2,1451606220,2015-12-31 23:57:00,BTC/USD,430.89,430.89,430.89,430.89,0.000000,0.000000
3,1451606160,2015-12-31 23:56:00,BTC/USD,430.89,430.89,430.89,430.89,0.116193,50.066229
4,1451606100,2015-12-31 23:55:00,BTC/USD,430.89,430.89,430.89,430.89,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
525594,1420070700,2015-01-01 00:05:00,BTC/USD,320.50,320.51,320.49,320.50,8.255000,2645.727500
525595,1420070640,2015-01-01 00:04:00,BTC/USD,320.98,320.98,320.43,320.43,2.843880,911.264468
525596,1420070580,2015-01-01 00:03:00,BTC/USD,321.00,321.00,321.00,321.00,2.000000,642.000000
525597,1420070520,2015-01-01 00:02:00,BTC/USD,321.00,321.00,321.00,321.00,1.413333,453.679999


In [58]:
def readCSV(file):
    df = pd.read_csv(file,usecols = ["unix","high","low","open","Volume BTC","Volume USD","close"])
    df.rename(columns={'unix': 'time', 'Volume BTC': 'volumeto', 'Volume USD': 'volumefrom'}, inplace=True)

    titles = ["time","high" ,"low","open" ,"volumefrom","volumeto" ,"close"]

    df = df[titles]

    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='s')
    
    return df

In [64]:
hist = readCSV(csvfiles[0])
target_col = 'close'
hist

,high,low,open,volumefrom,volumeto,close
time,,,,,,
2015-12-31 23:59:00,430.89,430.89,430.89,0.000000,0.000000,430.89
2015-12-31 23:58:00,430.89,429.77,429.77,94.454380,0.219208,430.89
2015-12-31 23:57:00,430.89,430.89,430.89,0.000000,0.000000,430.89
2015-12-31 23:56:00,430.89,430.89,430.89,50.066229,0.116193,430.89
2015-12-31 23:55:00,430.89,430.89,430.89,0.000000,0.000000,430.89
...,...,...,...,...,...,...
2015-01-01 00:05:00,320.51,320.49,320.50,2645.727500,8.255000,320.50
2015-01-01 00:04:00,320.98,320.43,320.98,911.264468,2.843880,320.43
2015-01-01 00:03:00,321.00,321.00,321.00,642.000000,2.000000,321.00


In [60]:
def train_test_split(df, test_size=0.2):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

In [61]:
train, test = train_test_split(hist, test_size=0.1)

In [62]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('price [CAD]', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [63]:
line_plot(train[target_col], test[target_col], 'training', 'test', title='')

NameError: name 'plt' is not defined

In [ ]:
def normalise_zero_base(df):
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    return (df - df.min()) / (data.max() - df.min())

In [ ]:
def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [ ]:
def prepare_data(df, target_col, window_len=10, zero_base=True, test_size=0.1):
    train_data, test_data = train_test_split(df, test_size=test_size)
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test

In [ ]:
def build_lstm_model(input_data, output_size, neurons=300, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:
np.random.seed(42)
window_len = 5
test_size = 0.1
zero_base = True
lstm_neurons = 300
epochs = 50
batch_size = 36
loss = 'mse'
dropout = 0.2
optimizer = 'adam'

In [ ]:
train, test, X_train, X_test, y_train, y_test = prepare_data(hist, target_col, window_len=window_len, zero_base=zero_base, test_size=test_size)

In [ ]:
model = build_lstm_model(X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,optimizer=optimizer)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
print(type(model))
print(type(history))
print(type(history.history))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'],'r',linewidth=2, label='Train loss')
plt.plot(history.history['val_loss'], 'g',linewidth=2, label='Validation loss')
plt.title('LSTM')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.show()

In [ ]:
targets = test[target_col][window_len:]
preds = model.predict(X_test).squeeze()
mean_absolute_error(preds, y_test)
print(type(targets))
print(type(preds))
mean_absolute_error(preds, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
MAE=mean_squared_error(preds, y_test)
MAE

In [ ]:
from sklearn.metrics import r2_score
R2=r2_score(y_test, preds)
R2

In [ ]:
preds = test[target_col].values[:-window_len] * (preds + 1)
preds = pd.Series(index=targets.index, data=preds)
line_plot(targets, preds, 'actual', 'prediction', lw=3)